In [58]:
from pymongo import MongoClient, 
import pandas as pd

client = MongoClient(host='localhost', port=27017)
db = client['amensch']

runs = db['runs']


def first_trial():
    # Build dataframe
    results = runs.aggregate([{"$match": {'experiment.name': 'decomposition',
                                          '_id': {"$gte": 374, "$lte": 380}}},
                              {"$sort": {'_id': -1}},
                              {"$skip": 1},
                              {"$limit": 6},
                              {"$project":
                                  {
                                      'alpha': '$config.alpha',
                                      'n_components': '$config.n_components',
                                      'dec_score': '$info.dec_score',
                                      'dec_final_score': '$info.dec_final_score',
                                      'artifacts': '$artifacts',
                                  }
                              },
                              {"$unwind": "$artifacts"},
                              {"$match": {'artifacts.name': {"$regex": ".*components.*.nii.gz"}}},
                              {"$project":
                                  {
                                      'components_name': '$artifacts.name',
                                      'components_file_id': '$artifacts.file_id',
                                      'alpha': 1,
                                      'n_components': 1,
                                      'dec_score': 1,
                                      'dec_final_score': 1,
                                  }
                              },
                              ])

    return pd.DataFrame(list(results))

In [55]:
df =first_trial()

In [57]:
import os
from os.path import expanduser

ids = df['components_file_id']
output_dir = expanduser('~/output/decomposition')
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
for this_id in ids:

0    58a38d81611b0a5f8558e950
1    58a38ac7611b0a5f8458e950
2    58a38ac6611b0a5f8358e950
3    58a38a3d611b0a5f8258e950
4    58a4334f611b0a5f8258e966
5    58a4336a611b0a5f8258e97b
6    58a38a3c611b0a5f8158e950
7    58a4334f611b0a5f8158e959
8    58a4336a611b0a5f8158e961
Name: components_file_id, dtype: object